In [0]:
#!pip3 install torch==1.4.0+cu92 torchvision==0.5.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

In [0]:
BATCH_SIZE = 12
 
#transformerer om til Tensor - slags Numpy-list
transform = transforms.Compose([transforms.ToTensor()])

# laster ned MNIST datasettet som torch har liggende inne. lagrer det i trainset var.  
#Transform = none?
trainset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
# loader dataset, med gitt Batch size.  
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

# nå laster vi ned test set, over var det train.
testset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

In [0]:
class NnDigitsClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        super(NnDigitsClassifier, self).__init__()

        # 28x28x1 => 26x26x32
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=12, kernel_size=5)
        self.pool = nn.MaxPool2d(2, 2)
        self.d1 = nn.Linear(12 * 4 * 4, 120)
        self.d2 = nn.Linear(120, 60)
        self.out = nn.Linear(60, 10)


    def forward(self, x):

        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)

        x = x.reshape(-1,12*4*4)
        x = self.d1(x)
        x = F.relu(x)

        
        logits = self.d2(x)
        out = F.softmax(logits, dim=1)
        #out = self.out(x)
        return out

In [0]:
## test the model with 1 batch
model = NnDigitsClassifier()
for images, labels in trainloader:
    out = model(images)
    break

In [0]:
learning_rate = 0.001
num_epochs = 5

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = NnDigitsClassifier()
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## compute accuracy
def get_accuracy(logit, target, batch_size):
    ''' Obtain accuracy for training round '''
    logit = logit.view(BATCH_SIZE, -1)
    corrects = (torch.max(logit, 1)[1].view(target.size()).data == target.data).sum()
    accuracy = 100.0 * corrects/batch_size
    return accuracy.item()

In [134]:
for epoch in range(num_epochs):
    train_running_loss = 0.0
    train_acc = 0.0

    model = model.train()

    ## training step
    for i, (images, labels) in enumerate(trainloader):
        
        images = images.to(device)
        labels = labels.to(device)

        ## forward + backprop + loss
        logits = model(images)

        logits = logits.view(BATCH_SIZE, -1)
        loss = criterion(logits, labels)
        optimizer.zero_grad()
        loss.backward()

        ## update model params
        optimizer.step()

        train_running_loss += loss.detach().item()
        train_acc += get_accuracy(logits, labels, BATCH_SIZE)
    
    model.eval()
    print('Epoch: %d | Loss: %.4f | Train Accuracy: %.2f' \
          %(epoch, train_running_loss / i, train_acc/i)) 

Epoch: 0 | Loss: 5.6553 | Train Accuracy: 32.58
Epoch: 1 | Loss: 5.5586 | Train Accuracy: 59.20
Epoch: 2 | Loss: 5.5454 | Train Accuracy: 76.31
Epoch: 3 | Loss: 5.5369 | Train Accuracy: 77.45
Epoch: 4 | Loss: 5.5309 | Train Accuracy: 78.07


In [135]:
test_acc = 0.0
for i, (images, labels) in enumerate(testloader, 0):
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    if (outputs.shape[0] == BATCH_SIZE * 9):
      #print(outputs.shape[0])
      test_acc += get_accuracy(outputs, labels, BATCH_SIZE)
        
print('Test Accuracy: %.2f'%( test_acc/i))

Test Accuracy: 77.35
